In [1]:
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests
import pandas as pd
import numpy as np
# import acquire1
import re
import unicodedata
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords

from env import github_token, github_username

/usr/local/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


From reading the script documentation it seems that the acquire is taken care of and pulls in a JSON data file that hold all of the data.

From here I will need to import the results and see what I can do with them

In reviewing the material inside of the JSON file I am going to need to declutter the script.
- There are many free floating characters present 
- Turn this clean data into a pandas dataframe

<a href="https://ds.codeup.com/nlp/explore/">This chapter</a>

In [2]:
df = pd.read_json("data.json")
df

,repo,language,readme_contents
0,dickreuter/Poker,Python,"Deep mind pokerbot for pokerstars, partypoker ..."
1,pret/pokered,Assembly,# Pokémon Red and Blue [![Build Status][ci-bad...
2,ishikota/PyPokerEngine,Python,# PyPokerEngine\n\n[![Build Status](https://tr...
3,andrewprock/pokerstove,C++,PokerStove\n==========\n\n[![OpenHUB Metrics](...
4,pokerregion/poker,Python,Poker framework for Python\n==================...
...,...,...,...
495,IMGameDesigner/PokerGame,JavaScript,### 满意请给个star\n### [1.0原版js](https://github.co...
496,houen/PokerFace,ActionScript,This [Actionscript Flash poker (5-card & Texas...
497,ngocbd/pokerserver,Java,# pokerserver\n[![Codacy Badge](https://api.co...
498,mikaelbr/AI-Poker-Player,Python,# AI Poker Player\n===========================...


In [3]:
df.language.value_counts()

JavaScript          103
Python               87
Java                 59
C++                  31
C#                   26
TypeScript           24
Ruby                 17
C                    15
PHP                  14
Jupyter Notebook     11
Assembly             11
Go                   11
Swift                 7
Elixir                5
HTML                  5
Objective-C           4
Erlang                4
ActionScript          4
CSS                   4
Clojure               4
Rust                  4
Lua                   3
Scala                 3
Haskell               2
Pascal                2
Dart                  2
Vue                   2
CoffeeScript          2
Kotlin                2
Shell                 2
Svelte                2
Roff                  2
F#                    1
CMake                 1
Julia                 1
Elm                   1
Groovy                1
Apex                  1
ShaderLab             1
Name: language, dtype: int64

Restarted approach to getting all of the text formatted

In [4]:
df.readme_contents

0      Deep mind pokerbot for pokerstars, partypoker ...
1      # Pokémon Red and Blue [![Build Status][ci-bad...
2      # PyPokerEngine\n\n[![Build Status](https://tr...
3      PokerStove\n==========\n\n[![OpenHUB Metrics](...
4      Poker framework for Python\n==================...
                             ...                        
495    ### 满意请给个star\n### [1.0原版js](https://github.co...
496    This [Actionscript Flash poker (5-card & Texas...
497    # pokerserver\n[![Codacy Badge](https://api.co...
498    # AI Poker Player\n===========================...
499    # Planning Poker\n[![Codacy Badge](https://api...
Name: readme_contents, Length: 500, dtype: object

In [6]:
type(df.readme_contents)

pandas.core.series.Series

Error code above indicates that re.sub can only take in string or byte-like objects:

Can run through a for loop with the chnages being applied

In [11]:
nltk.download('words')
words = set(nltk.corpus.words.words())

[nltk_data] Downloading package words to
[nltk_data]     /Users/lelouchlamprerouge/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [17]:
def show_all_matches(regexes, subject, re_length=6):
    print('Sentence:')
    print()
    print('    {}'.format(subject))
    print()
    print(' regexp{} | matches'.format(' ' * (re_length - 6)))
    print(' ------{} | -------'.format(' ' * (re_length - 6)))
    for regexp in regexes:
        fmt = ' {:<%d} | {!r}' % re_length
        matches = re.findall(regexp, subject)
        if len(matches) > 8:
            matches = matches[:8] + ['...']
        print(fmt.format(regexp, matches))


So from here I can find specifically the https website code,

that would be semantics some of these misspelled words could clue us in on where to go next

In [19]:
def prepare_blogs(Series):
    """ This function takes in a Series and applies a series of cleaning functions before stemming and lemmatizing the text.
    Args: Series
    Function: .lower, normalize, remove non-ASCII, stem, lem
    Returns: Cleaned, stemmed, lemmatized Series in a DF, along with original text"""
    original_content = []
    clean_content = []
    stemmed_content = []
    lemmed_content = []
    blogs_dict = {'content': original_content,
    'cleaned_content': clean_content,
    'stemmed_content': stemmed_content,
    'lemmed_content': lemmed_content}
    for i in range(0, len(Series)):
        content = Series[i]
        # add unaltered text to list 'original_content'
        original_content.append(content)
        # convert to lower case
        content = content.lower()
        # remove accented characters
        # unicode: removes character encoding incosistencies
        # .encode: converts resulting str chars to ASCII set. ignore errors will drop no ASCII chars
        # .decode turns the bytes object back into an str
        content = unicodedata.normalize('NFKD', content)\
        .encode('ascii', 'ignore')\
        .decode('utf-8', 'ignore')
        # remove non-standard alphanumeric characters
        content = re.sub(r"[^a-z0-9'\s]", '', content)
        # tokenization:
        # break words and punctuation left over into discrete units
        tokenizer = nltk.tokenize.ToktokTokenizer()
        tokenizer.tokenize(content, return_str=True)
        # add the tokenized text to the list 'clean_content'
        clean_content.append(content)
        # stems are the base of words, call: calls, called, calling
        # to stem, create the object first:
        ps = nltk.porter.PorterStemmer()
        # then apply to all words in the article
        stems = [ps.stem(word) for word in content.split()]
        content_stemmed = ' '.join(stems)
        #add stemmed output to list:
        stemmed_content.append(content_stemmed)
        # lemmatizing: reduces the word by removing the suffix (if applicable), but leaves a lexi cor word
        # how to lemmatize:
        wnl = nltk.stem.WordNetLemmatizer()
        lemmas = [wnl.lemmatize(word) for word in content.split()]
        content_lemmatized = ' '.join(lemmas)
        # add lemmed content to list:
        lemmed_content.append(content_lemmatized)
    df = pd.DataFrame(blogs_dict)
    return df

In [20]:
cleaned_df = prepare_blogs(df.readme_contents)

In [21]:
cleaned_df

,content,cleaned_content,stemmed_content,lemmed_content
0,"Deep mind pokerbot for pokerstars, partypoker ...",deep mind pokerbot for pokerstars partypoker a...,deep mind pokerbot for pokerstar partypok and ...,deep mind pokerbot for pokerstars partypoker a...
1,# Pokémon Red and Blue [![Build Status][ci-bad...,pokemon red and blue build statuscibadgeci\n\...,pokemon red and blue build statuscibadgeci thi...,pokemon red and blue build statuscibadgeci thi...
2,# PyPokerEngine\n\n[![Build Status](https://tr...,pypokerengine\n\nbuild statushttpstravisciorg...,pypokerengin build statushttpstravisciorgishik...,pypokerengine build statushttpstravisciorgishi...
3,PokerStove\n==========\n\n[![OpenHUB Metrics](...,pokerstove\n\n\nopenhub metricshttpswwwopenhub...,pokerstov openhub metricshttpswwwopenhubnetppo...,pokerstove openhub metricshttpswwwopenhubnetpp...
4,Poker framework for Python\n==================...,poker framework for python\n\n\ntravis coveral...,poker framework for python travi coveral pytho...,poker framework for python travis coverall pyt...
...,...,...,...,...
495,### 满意请给个star\n### [1.0原版js](https://github.co...,star\n 10jshttpsgithubcomimgamedesignerpokerg...,star 10jshttpsgithubcomimgamedesignerpokergame...,star 10jshttpsgithubcomimgamedesignerpokergame...
496,This [Actionscript Flash poker (5-card & Texas...,this actionscript flash poker 5card texas hol...,thi actionscript flash poker 5card texa holdem...,this actionscript flash poker 5card texas hold...
497,# pokerserver\n[![Codacy Badge](https://api.co...,pokerserver\ncodacy badgehttpsapicodacycompro...,pokerserv codaci badgehttpsapicodacycomproject...,pokerserver codacy badgehttpsapicodacycomproje...
498,# AI Poker Player\n===========================...,ai poker player\n\ninitial commit for the ai ...,ai poker player initi commit for the ai poker ...,ai poker player initial commit for the ai poke...


In [25]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   content          500 non-null    object
 1   cleaned_content  500 non-null    object
 2   stemmed_content  500 non-null    object
 3   lemmed_content   500 non-null    object
dtypes: object(4)
memory usage: 15.8+ KB


In [26]:
cleaned_df.lemmed_content[0].split()

['deep',
 'mind',
 'pokerbot',
 'for',
 'pokerstars',
 'partypoker',
 'and',
 'ggpoker',
 'this',
 'pokerbot',
 'play',
 'automatically',
 'on',
 'pokerstars',
 'partypoker',
 'and',
 'gg',
 'poker',
 'any',
 'other',
 'table',
 'can',
 'be',
 'mapped',
 'a',
 'well',
 'it',
 'work',
 'with',
 'image',
 'recognition',
 'montecarlo',
 'simulation',
 'and',
 'a',
 'basic',
 'genetic',
 'algorithm',
 'the',
 'mouse',
 'is',
 'moved',
 'automatically',
 'and',
 'the',
 'bot',
 'can',
 'potentially',
 'play',
 'for',
 'hour',
 'based',
 'on',
 'a',
 'large',
 'number',
 'of',
 'parameter',
 'you',
 'can',
 'download',
 'the',
 'binary',
 'and',
 'run',
 'the',
 'executable',
 'directly',
 'from',
 'here',
 'httpssourceforgenetprojectspartypokerpokerstarspokerbot',
 'running',
 'the',
 'bot',
 'partypoker',
 'use',
 'fast',
 'forward',
 'table',
 'select',
 'official',
 'partypoker',
 'table',
 'scraper',
 'figure',
 'docpartypokergif',
 'pokerstars',
 'use',
 'zoom',
 'table',
 'select',
 '

In [68]:
def splitwords(Series):
        terms_list= []
        frequencey_list = []
        augmented_frequency_list = []
        split_word_dict = {'terms_list': terms_list,
        'frequencey_list': frequencey_list,
        'augmented_frequency_list': augmented_frequency_list,}
        for i in range(0, len(Series)):
                word_list = Series[i]
                for word in word_list:
                        term = word 
                        terms_list.append(term)
                        frequency = word_list.value_counts() / word_list.value_counts().sum()
                        frequencey_list.append(frequency)
                        augmented_frequency = frequency / frequency.max()
                        augmented_frequency_list.append(augmented_frequency)
        return pd.DataFrame(split_word_dict)

In [28]:
new_frame = cleaned_df.lemmed_content.to_frame()
new_frame

,lemmed_content
0,deep mind pokerbot for pokerstars partypoker a...
1,pokemon red and blue build statuscibadgeci thi...
2,pypokerengine build statushttpstravisciorgishi...
3,pokerstove openhub metricshttpswwwopenhubnetpp...
4,poker framework for python travis coverall pyt...
...,...
495,star 10jshttpsgithubcomimgamedesignerpokergame...
496,this actionscript flash poker 5card texas hold...
497,pokerserver codacy badgehttpsapicodacycomproje...
498,ai poker player initial commit for the ai poke...


s = pd.Series(['A', 'B', 'C'])

for index, value in s.items():

    print(f"Index : {index}, Value : {value}")
Index : 0, Value : A

Index : 1, Value : B

Index : 2, Value : C

In [45]:
new_frame.lemmed_content.apply(lambda r: r.split())

0      [deep, mind, pokerbot, for, pokerstars, partyp...
1      [pokemon, red, and, blue, build, statuscibadge...
2      [pypokerengine, build, statushttpstravisciorgi...
3      [pokerstove, openhub, metricshttpswwwopenhubne...
4      [poker, framework, for, python, travis, covera...
                             ...                        
495    [star, 10jshttpsgithubcomimgamedesignerpokerga...
496    [this, actionscript, flash, poker, 5card, texa...
497    [pokerserver, codacy, badgehttpsapicodacycompr...
498    [ai, poker, player, initial, commit, for, the,...
499    [planning, poker, codacy, badgehttpsapicodacyc...
Name: lemmed_content, Length: 500, dtype: object

In [48]:
ee = new_frame.lemmed_content.apply(lambda r: r.split())

In [87]:
type(ee)

pandas.core.series.Series

In [86]:
type(ee[0])

list

In [78]:
ee

0      [deep, mind, pokerbot, for, pokerstars, partyp...
1      [pokemon, red, and, blue, build, statuscibadge...
2      [pypokerengine, build, statushttpstravisciorgi...
3      [pokerstove, openhub, metricshttpswwwopenhubne...
4      [poker, framework, for, python, travis, covera...
                             ...                        
495    [star, 10jshttpsgithubcomimgamedesignerpokerga...
496    [this, actionscript, flash, poker, 5card, texa...
497    [pokerserver, codacy, badgehttpsapicodacycompr...
498    [ai, poker, player, initial, commit, for, the,...
499    [planning, poker, codacy, badgehttpsapicodacyc...
Name: lemmed_content, Length: 500, dtype: object

In [80]:
# document = 'Mary had a little lamb, a little lamb, a little lamb.'

# # clean up the text
# document = document.lower().replace(',', '').replace('.', '')
# # transform into a series
# words = pd.Series(document.split())

# From the Series we can extract the value_counts, which is our raw count
# for term frequency. Once we have the raw counts, we can calculate the
# other measures.
(pd.DataFrame({'raw_count': new_frame.value_counts()})
 .assign(frequency=lambda df: df.raw_count / df.raw_count.sum())
 .assign(augmented_frequency=lambda df: df.frequency / df.frequency.max()))


raw_count  frequency  \
lemmed_content                                                             
                                                           39      0.078   
ironhack logohttpsiimgurcom1qgrnnwpng lab calcu...          4      0.008   
poker                                                       2      0.004   
pokemon red and blue build statustravisbadgetra...          2      0.004   
unity3d licence badgelicence star badgestars fo...          2      0.004   
...                                                       ...        ...   
nodeangularsocketio planning poker planning pok...          1      0.002   
nlth poker agent using monte carlo simulation m...          1      0.002   
nlth poker agent using counterfactual regret mi...          1      0.002   
nextgenerationscrumpoker prototype for a new sc...          1      0.002   
zippysolver an open source solver developed by ...          1      0.002   

                                                    augmented_frequency  
lemmed_content                                                           
                                                               1.000000  
ironhack logohttpsiimgurcom1qgrnnwpng lab calcu...             0.102564  
poker                                                          0.051282  
pokemon red and blue build statustravisbadgetra...             0.051282  
unity3d licence badgelicence star badgestars fo...             0.051282  
...                                                                 ...  
nodeangularsocketio planning poker planning pok...             0.025641  
nlth poker agent using monte carlo simulation m...             0.025641  
nlth poker agent using counterfactual regret mi...             0.025641  
nextgenerationscrumpoker prototype for a new sc...             0.025641  
zippysolver an open source solver developed by ...             0.025641  

[454 rows x 3 columns]

In [92]:
#the solution will come for the apply method
#the problem pointed out with ee is that the string are in fact split but they return one string that has all these entries
#now then to make the most of the these lists
#I would need to make each word its own index point in a series 
#the problem here is that if I end  up splitting it this way I lose the index and we lose where these words come from
#this can
words
#words in this specific scenario are a pandas series

0       mary
1        had
2          a
3     little
4       lamb
5          a
6     little
7       lamb
8          a
9     little
10      lamb
dtype: object

word_counts = (pd.concat([all_freq, ham_freq, spam_freq], axis=1, sort=True)
                .set_axis(['all', 'ham', 'spam'], axis=1, inplace=False)
                .fillna(0)
                .apply(lambda s: s.astype(int)))

word_counts.head()
i can label spam and ham as i need to for programming languages 
the all count will tell me all of the instances of where it came

In [79]:
new_frame

,lemmed_content
0,deep mind pokerbot for pokerstars partypoker a...
1,pokemon red and blue build statuscibadgeci thi...
2,pypokerengine build statushttpstravisciorgishi...
3,pokerstove openhub metricshttpswwwopenhubnetpp...
4,poker framework for python travis coverall pyt...
...,...
495,star 10jshttpsgithubcomimgamedesignerpokergame...
496,this actionscript flash poker 5card texas hold...
497,pokerserver codacy badgehttpsapicodacycomproje...
498,ai poker player initial commit for the ai poke...


In [94]:
# for i in new_frame.lemmed_content:
word_list = new_frame.lemmed_content[0].split()

In [95]:
type(word_list)

list

In [103]:
words_list = pd.Series(new_frame.lemmed_content[0].split())

# From the Series we can extract the value_counts, which is our raw count
# for term frequency. Once we have the raw counts, we can calculate the
# other measures.
(pd.DataFrame({'raw_count': words_list.value_counts()})
 .assign(frequency=lambda df: df.raw_count / df.raw_count.sum())
 .assign(augmented_frequency=lambda df: df.frequency / df.frequency.max()))


,raw_count,frequency,augmented_frequency
the,169,0.072626,1.000000
to,87,0.037387,0.514793
a,54,0.023206,0.319527
and,50,0.021487,0.295858
you,45,0.019338,0.266272
...,...,...,...
read,1,0.000430,0.005917
learn,1,0.000430,0.005917
easiest,1,0.000430,0.005917
codebase,1,0.000430,0.005917


In [ ]:
labels = pd.concat([df.language.value_counts(),
                    df.language.value_counts(normalize=True)], axis=1)
labels.columns = ['n', 'percent']
labels
#same information as value_counts()

In [108]:
df.language.value_counts()

JavaScript          103
Python               87
Java                 59
C++                  31
C#                   26
TypeScript           24
Ruby                 17
C                    15
PHP                  14
Jupyter Notebook     11
Assembly             11
Go                   11
Swift                 7
Elixir                5
HTML                  5
Objective-C           4
Erlang                4
ActionScript          4
CSS                   4
Clojure               4
Rust                  4
Lua                   3
Scala                 3
Haskell               2
Pascal                2
Dart                  2
Vue                   2
CoffeeScript          2
Kotlin                2
Shell                 2
Svelte                2
Roff                  2
F#                    1
CMake                 1
Julia                 1
Elm                   1
Groovy                1
Apex                  1
ShaderLab             1
Name: language, dtype: int64

In [109]:
ADDITIONAL_STOPWORDS = ['r', 'u', '2', 'ltgt']

def clean(text):
    'A simple function to cleanup text data'
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = nltk.corpus.stopwords.words('english') + ADDITIONAL_STOPWORDS
    text = (unicodedata.normalize('NFKD', text)
             .encode('ascii', 'ignore')
             .decode('utf-8', 'ignore')
             .lower())
    words = re.sub(r'[^\w\s]', '', text).split()
    return [wnl.lemmatize(word) for word in words if word not in stopwords]


In [120]:
JavaScript = clean(' '.join(df[df.language == 'JavaScript'].readme_contents))
Java = clean(' '.join(df[df.language == 'Java'].readme_contents))
Cplusplus = clean(' '.join(df[df.language == 'C++'].readme_contents))
C_sharp = clean(' '.join(df[df.language == 'C#'].readme_contents))
TypeScript = clean(' '.join(df[df.language == 'TypeScript'].readme_contents))
Ruby = clean(' '.join(df[df.language == 'Ruby'].readme_contents))
Cee = clean(' '.join(df[df.language == 'C'].readme_contents))
PeeHpee = clean(' '.join(df[df.language == 'PHP'].readme_contents))
Jupyter_Notebook = clean(' '.join(df[df.language == 'Jupyter Notebook'].readme_contents))
Assembly = clean(' '.join(df[df.language == 'Assembly'].readme_contents))

all_words = clean(' '.join(df.readme_contents))


In [137]:
JavaScript_freq = pd.Series(JavaScript).value_counts()
Java_ferq = pd.Series(Java).value_counts()
Cplusplus_freq = pd.Series(Cplusplus).value_counts()
C_sharp_freq = pd.Series(C_sharp).value_counts()
TypeScript_freq = pd.Series(TypeScript).value_counts()
Ruby_freq = pd.Series(Ruby).value_counts()
Cee_freq = pd.Series(Cee).value_counts()
PeeHpee_freq = pd.Series(PeeHpee).value_counts()
Jupyter_Notebook_freq = pd.Series(Jupyter_Notebook).value_counts()
Assembly_freq = pd.Series(Assembly).value_counts()
all_freq = pd.Series(all_words).value_counts()

# all_freq.head(20)
# creating a wordcloud from here will yeild a visualization

In [ ]:
labels = pd.concat([df.language.value_counts(),
                    df.language.value_counts(normalize=True)], axis=1)
labels.columns = ['n', 'percent']
labels

In [138]:
word_counts = (pd.concat([all_freq, JavaScript_freq, Java_ferq, Cplusplus_freq, C_sharp_freq, TypeScript_freq, Ruby_freq, Cee_freq, PeeHpee_freq, Jupyter_Notebook_freq, Assembly_freq], axis=1, sort=True)
                .set_axis(['all_words', 'JavaScript', 'Java', 'C++', 'C#', 'TypeScript', 'Ruby', 'C', 'PHP', 'Jupyter Notebook', 'Assembly'], axis=1, inplace=False)
                .fillna(0)
                .apply(lambda s: s.astype(int)))

word_counts.head()

,all_words,JavaScript,Java,C++,C#,TypeScript,Ruby,C,PHP,Jupyter Notebook,Assembly
0,229,79,7,23,7,3,2,1,3,28,0
00,7,0,0,0,0,0,0,0,0,7,0
000,2,0,0,0,0,1,0,0,0,0,0
0000,1,1,0,0,0,0,0,0,0,0,0
000000,2,0,0,0,1,1,0,0,0,0,0


In [141]:
word_counts.sort_values(by='all_words', ascending=False).head(20)

,all_words,JavaScript,Java,C++,C#,TypeScript,Ruby,C,PHP,Jupyter Notebook,Assembly
poker,1163,208,148,42,86,51,59,45,30,48,0
player,1032,177,162,41,58,78,37,19,4,29,0
game,1013,167,145,58,65,12,15,18,17,30,1
hand,1007,222,71,86,83,38,36,82,16,21,0
card,989,259,84,71,46,23,14,40,23,50,1
run,662,201,52,42,27,43,29,26,1,5,0
1,547,152,42,39,34,25,17,13,9,37,0
use,447,88,63,41,8,25,18,15,5,6,0
code,416,113,47,19,17,9,31,14,6,8,3
install,403,100,12,24,10,27,30,9,4,0,0


Here is a huge win from here David will take the stopwords out and get this project going as for me I am going to need to repeat this process over for all of the rest of the corpus text

Snippets from the explore chapter both in the circulumn and excercise review

Ryan Mcall referenced his <a href="https://github.com/CodeupClassroom/jemison-nlp-exercises/blob/main/prepare_exercises.ipynb">exercise </a>

From here the problem is that if the words are split up what ways do we have to organize them? 

I feel like we can cross that bridge when we get there.

We might need to implement stopwords to get rid of non-useful words.
- the concern here is that words like "the, a, an" are gonna come up as the most often used
    - these do not yeild useful information
    - "tf and idf will account for the stopwords"